In [1]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
import unicodedata
import re
import utils as utils

In [ ]:
source_db = "landing"
source_table = "aena"
db_name = "trusted"
table_name = "f_ocupacion_hotelera"

In [2]:
spark = utils.create_context()

In [ ]:
@udf(StringType())
def normalize_text(text:str):

    if text is None:
        return ""
    
    # 1. Normalizar (separar letras de tildes)
    texto_normalizado = unicodedata.normalize('NFKD', text)
    
    # 2. Eliminar acentos (caracteres "combining")
    texto_sin_acentos = ''.join(
        c for c in texto_normalizado if not unicodedata.combining(c)
    )
    
    # 3. Eliminar puntuación (todo lo que no sea letra, número o espacio)
    texto_sin_puntuacion = re.sub(r'[^A-Za-z0-9\s]', '', texto_sin_acentos)
    
    # 4. Eliminar espacios extra
    texto_limpio = ' '.join(texto_sin_puntuacion.split())
    
    # 5. Convertir a mayúsculas
    return texto_limpio.upper()

In [ ]:

df = utils.read_iceberg_table(spark,db_name,table_name)

# convert each column to normalized type
df = (
    df
    .replace(float('nan'),None)
    .select(
        col("AÑO").alias("año"),
        col("MES").alias("mes"),
        normalize_text(col("PUNTO_TURISTICO")).alias("puntoTuristico"),
        normalize_text(col("LUGAR_RESIDENCIA")).alias("lugarResidencia"),
        normalize_text(col("PROVINCIA")).alias("provincia"),
        col("VIAJEROS").alias("viajeros").cast(IntegerType()),
        col("PERNOCTACIONES").alias("pernoctaciones").cast(IntegerType()),
        col("ESTANCIA_MEDIA").alias("estanciaMedia").cast(DoubleType()),
        col("ESTABLECIMIENTOS_ESTIMADOS").alias("establecimientosEstimados").cast(IntegerType()),
        col("HABITACIONES_ESTIMADAS").alias("habitacionesEstimadas").cast(IntegerType()),
        col("PLAZAS_ESTIMADAS").alias("plazasEstimadas").cast(IntegerType()),
        col("GRADO_OCUPA_PLAZAS").alias("gradoOcupaPlazas").cast(DoubleType()),
        col("GRADO_OCUPA_PLAZAS_FIN_SEMANA").alias("gradoOcupaPlazasFinSemana").cast(DoubleType()),
        col("GRADO_OCUPA_POR_HABITACIONES").alias("gradoOcupaPorHabitaciones").cast(DoubleType()),
        col("PERSONAL_EMPLEADO").alias("personalEmpleado").cast(IntegerType())
    )
)

# Filter out rows where all the mesures are null

condition = ~((col("viajeros").isNull()) & 
              (col("pernoctaciones").isNull()) & 
              (col("estanciaMedia").isNull()) & 
              (col("establecimientosEstimados").isNull()) & 
              (col("habitacionesEstimadas").isNull()) &
              (col("plazasEstimadas").isNull()) & 
              (col("gradoOcupaPlazas").isNull()) & 
              (col("gradoOcupaPlazasFinSemana").isNull()) & 
              (col("gradoOcupaPorHabitaciones").isNull()) & 
              (col("personalEmpleado").isNull())
              )

df = df.filter(condition=condition)

# Filter out also all the rows where the column viajeros < 0

df = df.filter(col("viajeros") < 0)

In [ ]:
utils.overwrite_iceberg_table(spark,df,db_name,table_name)

In [3]:
df = utils.read_iceberg_table(spark,"local_db","hoteles_punto_turistico")

df.show()

+----+---+----------------+--------------------+--------------------+--------+--------------+--------------+--------------------------+----------------------+----------------+------------------+-----------------------------+----------------------------+-----------------+
| AÑO|MES| PUNTO_TURISTICO|    LUGAR_RESIDENCIA|           PROVINCIA|VIAJEROS|PERNOCTACIONES|ESTANCIA_MEDIA|ESTABLECIMIENTOS_ESTIMADOS|HABITACIONES_ESTIMADAS|PLAZAS_ESTIMADAS|GRADO_OCUPA_PLAZAS|GRADO_OCUPA_PLAZAS_FIN_SEMANA|GRADO_OCUPA_POR_HABITACIONES|PERSONAL_EMPLEADO|
+----+---+----------------+--------------------+--------------------+--------+--------------+--------------+--------------------------+----------------------+----------------+------------------+-----------------------------+----------------------------+-----------------+
|2012|  1|           Adeje|Residentes en España|Santa Cruz de Ten...| 11779.0|       73680.0|          6.26|                       NaN|                   NaN|             NaN|         